# Task 2: Visualizing fire scars through false color
#### Author: Sofia Rodas

- Homework 4 GitHub repository: https://github.com/sofiiir/eds220-hwk4.git

## Data import and preparation

In [2]:
# Import packages
import os 
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr


os.environ['PROJ_LIB'] = '/opt/anaconda3/share/proj'


# ------ IMPORT DATASETS ------

fp = os.path.join('data', 'eaton_fire_perimeter', 'Eaton_Perimeter_20250121.shp')
eaton_fire = gpd.read_file(fp)

fp = os.path.join('data', 'palisades_fire_perimeter', 'Palisades_Perimeter_20250121.shp')
palisades_fire = gpd.read_file(fp)

fp = os.path.join('data', 'landsat8-2025-02-23-palisades-eaton.nc')
landsat = xr.open_dataset(fp)